Ссылка на репозиторий проекта:
https://github.com/andrew-tvorogov/practicum_masterskaya_1

## Рекомендации для инвесторов о целесообразности инвестирования в стартапы, на основе данных о стартапах, функционировавших в период с 1970 по 2018 годы.

<b>Заказчик исследования:</b>
- yandex practicum

<b>Источник данных:</b>
- псевдо-реальные (реальные данные в которые добавлена синтетическая составляющая) данные о стартапах, функционировавших в период с 1970 по 2018 годы

<b>Содержание данных:</b>
- данные о стартапах;

<b>Порядок проведения исследования:</b>
- загрузить данные;
- изучить данные;
- выполнить предобработку данных;
- полноценный разведочный анализ
    - провести исследовательский анализ;
    - провести корреляционный анализ признаков датасета;
- разработка новых синтетических признаков;
- проверка на мультиколлинеарность;
- отбор финального набора обучающих признаков;
- выбрать лучшую модель (выбор и обучение моделей);
    - создать пайплан обработки признаков;
    - создать пайплайн выбора лучшей модели.
- итоговая оценка качества предсказания лучшей модели;
- анализ важности ее признаков;
- сделать общий вывод на основе готовых моделей (подготовка отчета по исследованию).

<b>Задача:</b>
<p>Разработать решение, которое позволит предсказать какие стартапы закроются.</p>

<b>Цель исследования:</b>
- снижение рисков при инвестировании в стартап;
- получение прибыли;

<b>Описание данных:</b>

Данные доступны в четырёх файлах.
- ``kaggle_startups_train_28062024.csv`` - описание данных;

Файл ``kaggle_startups_train_28062024.csv``:
<pre>
<b>id</b> — описание поля.
<b>...</b> — описание поля.
<b>...</b> — описание поля.
</pre>

In [1]:
# установка пакетов
# SHapley Additive exPlanations
!pip install shap --quiet 

# correlation analyzer library
!pip install phik --quiet

In [2]:
# Загрузка библиотек
import pandas as pd
import matplotlib.pyplot as plt # для диаграмм
import math # для диаграмм
import numpy as np # для равномерного распределения значений на диаграмме
import seaborn as sns
import shap # оценка вклада признака
import phik

from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler,
    RobustScaler, 
    OneHotEncoder, 
    OrdinalEncoder)

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

from warnings import simplefilter

In [3]:
# Функции

In [4]:
# инициализация глобальных переменных
RANDOM_STATE = 42 # Deep Thought number
TEST_SIZE = 0.25 # размер выборки

In [5]:
# настройки
pd.options.display.max_columns = None # необходимо чтобы head отобразил все колонки
plt.rcParams['figure.figsize'] = [15, 3] # размеры графика по умолчанию
sns.set(rc={'figure.figsize':(15, 10)}) # размеры графика seaborn по умолчанию
pd.set_option('display.float_format', '{:.6f}'.format) # отобразит в колонках округленные данные до 6-ти знаков

simplefilter(action='ignore', category=FutureWarning) # уберёт предупреждение "Set `keepdims`

## Загрузка и изучение данных

### Загрузка

In [6]:
# загрузка данных в датасеты
try:
    startups_train = pd.read_csv('./datasets/kaggle_startups_train_28062024.csv')
    startups_test = pd.read_csv('./datasets/kaggle_startups_test_28062024.csv')
    startups_submit = pd.read_csv('./datasets/kaggle_startups_sample_submit_28062024.csv')
    #money = pd.read_csv('./datasets/money.csv', sep=';', decimal=",")    
except:
    print("Ошибка. Данные не загружены")

In [7]:
startups_train

,name,category_list,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,first_funding_at,last_funding_at,closed_at
0,Lunchgate,Online Reservations|Restaurants,828626.000000,operating,CHE,25,Zurich,Zürich,2,2009-10-17,2011-05-01,2014-12-01,NaN
1,EarLens,Manufacturing|Medical|Medical Devices,42935019.000000,operating,USA,CA,SF Bay Area,Redwood City,4,2005-01-01,2010-05-04,2014-02-25,NaN
2,Reviva Pharmaceuticals,Biotechnology,35456381.000000,operating,USA,CA,SF Bay Area,San Jose,3,2006-01-01,2012-08-20,2014-07-02,NaN
3,Sancilio and Company,Health Care,22250000.000000,operating,NaN,NaN,NaN,NaN,3,2004-01-01,2011-09-01,2014-07-18,NaN
4,WireTough Cylinders,Manufacturing,NaN,operating,USA,VA,VA - Other,Bristol,1,2010-05-12,2012-02-01,2012-02-01,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52511,Videostream,Entertainment,NaN,operating,CAN,ON,Toronto,Kitchener,1,2012-01-01,2014-03-01,2014-03-01,NaN
52512,Hello Curry,Hospitality,500000.000000,operating,IND,2,Hyderabad,Hyderabad,1,2013-08-25,2014-03-07,2014-03-07,NaN
52513,Taskforce,Email|Messaging|Productivity Software,50000.000000,operating,USA,CA,SF Bay Area,San Francisco,3,2010-07-01,2009-06-14,2011-01-01,NaN
52514,NetScaler,Security,13000000.000000,operating,USA,CA,SF Bay Area,San Jose,6,1997-12-01,1998-11-30,2004-03-01,NaN
